In [1]:
import json
import glob
import datetime
import pandas as pd
from scipy import optimize

In [2]:
data = {file[-6:]:json.load(open(file)) for file in glob.glob('history*')}

In [3]:
data['000071'][0]

{'adjustTxnDate': 1510675200000,
 'adjustmentId': 85,
 'fund': {'canBuy': True,
  'canRedeem': True,
  'fundCode': '000071',
  'fundInvestType': '1',
  'fundName': '华夏恒生ETF联接',
  'isEtf': False,
  'isIndex': True,
  'isLof': False,
  'isPeriod': False,
  'isQdii': True,
  'onSale': True,
  'riskLevel': 3,
  'valDisBy': 'nav'},
 'nav': 1.5077,
 'navDate': 1510675200000,
 'orderCode': '024',
 'tradeUnit': 1,
 'variety': '恒生'}

In [4]:
len(data['000071'])

10

# 计算交易记录

In [5]:
def fmt_date(timestamp):
    return datetime.datetime.fromtimestamp(timestamp/1000).date()

def buy(code, order):
    status[code]['unit'] += order['tradeUnit']
    fund_num = int(unit_money / order['nav'])
    status[code]['num'] += fund_num
    transcations.append({'date': fmt_date(order['navDate']),
                         'code': code,
                         'unit': -order['tradeUnit'],
                         'money': -fund_num * order['nav']})


def sell(code, order):
    status[code]['unit'] -= order['tradeUnit']
    fund_num = int(status[code]['num'] / status[code]['unit']) * order['tradeUnit']
    status[code]['num'] -= fund_num
    transcations.append({'date': fmt_date(order['navDate']),
                         'code': code,
                         'unit': order['tradeUnit'],
                         'money': fund_num * order['nav']})

In [6]:
def update(code, order):
    status.setdefault(code, {'code':code, 'unit':0,'num':0})
    
    if order['orderCode'] == '022':
        buy(code, order)
    elif order['orderCode'] == '024':
        sell(code, order)
    else:
        print('unknown order code', code, order)

In [7]:
unit_money = 1000  # 每份 1000 元
status = {}  # 每个基金的份数和基金份额
transcations = []  # 每次交易的份数和金额

for k, v in data.items():
    for vv in v[::-1]:
        update(k, vv)

In [8]:
print('current unit:',sum(v['unit'] for v in status.values()))
print(len(transcations),'transcations')

current unit: 71
73 transcations


# 添加最新净值

In [9]:
for fund in json.load(open('plan.json'))['composition']:
    if fund['planUnit'] > 0 and 'fund' in fund:
        fund_status = status[fund['fund']['fundCode']]
        fund_status['date'] = fmt_date(fund['navDate'])
        fund_status['net'] = fund['nav']
        fund_status['value'] = fund['nav'] * fund_status['num']

# 转换为 DataFrame

In [10]:
status_df = pd.DataFrame(list(status.values())).sort_values('unit', ascending=False)
status_df = status_df[['code', 'unit', 'num', 'date', 'net', 'value']]
transcations_df = pd.DataFrame(transcations).sort_values('date')
transcations_df = transcations_df[['date', 'code', 'unit', 'money']]

In [11]:
status_df

,code,unit,num,date,net,value
1,000071,8,6118,2018-01-08,1.5711,9611.9898
10,001180,7,8829,2018-01-08,0.8642,7630.0218
3,000478,7,2488,2018-01-08,2.4412,6073.7056
5,000968,7,6286,2018-01-08,1.0934,6873.1124
6,001051,7,9815,2018-01-08,0.9880,9697.2200
11,003376,6,6371,2018-01-08,0.9265,5902.7315
14,100032,6,4743,2018-01-08,1.3810,6550.0830
9,001064,5,6582,2018-01-08,0.7790,5127.3780
7,001052,3,4794,2018-01-08,0.6870,3293.4780
18,270048,2,1688,2018-01-08,1.1790,1990.1520


In [12]:
transcations_df

,date,code,unit,money
25,2015-08-03,001051,-1,-999.1910
1,2015-09-01,000071,-1,-999.4710
26,2015-09-08,001051,-1,-999.3060
2,2015-11-02,000071,-1,-999.1040
3,2015-12-08,000071,-1,-999.9080
67,2015-12-14,160416,-1,-999.4010
4,2015-12-30,000071,-2,-999.3060
5,2016-01-11,000071,-1,-999.9840
27,2016-01-11,001051,-1,-999.3600
28,2016-02-01,001051,-1,-999.4530


# XIRR

In [13]:
# https://stackoverflow.com/questions/8919718/financial-python-library-that-has-xirr-and-xnpv-function
def xnpv(rate, values, dates):
    '''Equivalent of Excel's XNPV function.

    >>> from datetime import date
    >>> dates = [date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)]
    >>> values = [-10000, 20, 10100]
    >>> xnpv(0.1, values, dates)
    -966.4345...
    '''
    if rate <= -1.0:
        return float('inf')
    d0 = dates[0]    # or min(dates)
    return sum([ vi / (1.0 + rate)**((di - d0).days / 365.0) for vi, di in zip(values, dates)])

def xirr(values, dates):
    '''Equivalent of Excel's XIRR function.

    >>> from datetime import date
    >>> dates = [date(2010, 12, 29), date(2012, 1, 25), date(2012, 3, 8)]
    >>> values = [-10000, 20, 10100]
    >>> xirr(values, dates)
    0.0100612...
    '''
    try:
        return optimize.newton(lambda r: xnpv(r, values, dates), 0.0)
    except RuntimeError:    # Failed to converge?
        return optimize.brentq(lambda r: xnpv(r, values, dates), -1.0, 1e10)

In [14]:
dates = list(transcations_df['date'])
values = list(transcations_df['money'])
dates.append(status_df['date'].max())
values.append(status_df['value'].sum())

In [15]:
print('年化收益率:', xirr(values, dates))
print('项目开始时间:', dates[0])
print('截至 %s 已运行 %d 天' % (dates[-1], (dates[-1]-dates[0]).days))

年化收益率: 0.160825712598
项目开始时间: 2015-08-03
截至 2018-01-08 已运行 889 天


# 每月投入

In [16]:
transcations_df['month'] = transcations_df['date'].apply(lambda x: x.strftime('%Y%m'))

In [17]:
transcations_df.groupby('month')[['unit', 'money']].sum()

,unit,money
month,,
201508,-1,-999.1910
201509,-2,-1998.7770
201511,-1,-999.1040
201512,-4,-2998.6150
201601,-2,-1999.3440
201602,-4,-2998.2441
201603,-3,-2999.5500
201604,-2,-1999.5520
201605,-2,-1999.4130


# 输出CSV

In [18]:
status_df.to_csv('status.csv', index=False)
transcations_df.to_csv('transcations.csv', index=False)